In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import functions as F
import hashlib
from pyspark.ml.feature import StringIndexer
from datetime import datetime
from pyspark.sql.functions import UserDefinedFunction, col
from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf

In [0]:
df = spark.read.csv('/FileStore/tables/Australia_GP.csv', sep =',', header = True, inferSchema = True)
display(df)
print(f"DF COUNT: {df.count()}")

_c0,Time,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,Compound,TyreLife,FreshTyre,LapStartTime,Team,Driver,TrackStatus,IsAccurate,LapStartDate
0,0 days 01:03:43.368000,16,null,1,1,0 days 00:24:52.909000,null,null,0 days 00:00:18.645000,0 days 00:00:36.781000,null,0 days 01:03:06.669000,0 days 01:03:43.458000,261.0,301.0,277.0,277.0,MEDIUM,1.0,true,0 days 01:02:13.396000,Ferrari,LEC,1,false,2022-04-10T05:03:13.407+0000
1,0 days 01:05:08.114000,16,0 days 00:01:24.746000,2,1,null,null,0 days 00:00:29.566000,0 days 00:00:18.608000,0 days 00:00:36.572000,0 days 01:04:12.934000,0 days 01:04:31.542000,0 days 01:05:08.114000,262.0,293.0,280.0,280.0,MEDIUM,2.0,true,0 days 01:03:43.368000,Ferrari,LEC,2,true,2022-04-10T05:04:43.379+0000
2,0 days 01:06:51.459000,16,0 days 00:01:43.345000,3,1,null,null,0 days 00:00:29.279000,0 days 00:00:24.719000,0 days 00:00:49.347000,0 days 01:05:37.393000,0 days 01:06:02.112000,0 days 01:06:51.459000,268.0,157.0,241.0,247.0,MEDIUM,3.0,true,0 days 01:05:08.114000,Ferrari,LEC,264,false,2022-04-10T05:06:08.125+0000
3,0 days 01:09:06.561000,16,0 days 00:02:15.102000,4,1,null,null,0 days 00:00:42.800000,0 days 00:00:30.843000,0 days 00:01:01.459000,0 days 01:07:34.259000,0 days 01:08:05.102000,0 days 01:09:06.561000,138.0,171.0,229.0,null,MEDIUM,4.0,true,0 days 01:06:51.459000,Ferrari,LEC,4,false,2022-04-10T05:07:51.470+0000
4,0 days 01:11:21.497000,16,0 days 00:02:14.936000,5,1,null,null,0 days 00:00:45.666000,0 days 00:00:30.394000,0 days 00:00:58.876000,0 days 01:09:52.227000,0 days 01:10:22.621000,0 days 01:11:21.497000,177.0,200.0,221.0,134.0,MEDIUM,5.0,true,0 days 01:09:06.561000,Ferrari,LEC,4,false,2022-04-10T05:10:06.572+0000
5,0 days 01:13:46.450000,16,0 days 00:02:24.953000,6,1,null,null,0 days 00:00:45.174000,0 days 00:00:31.308000,0 days 00:01:08.471000,0 days 01:12:06.671000,0 days 01:12:37.979000,0 days 01:13:46.450000,185.0,136.0,280.0,291.0,MEDIUM,6.0,true,0 days 01:11:21.497000,Ferrari,LEC,4,false,2022-04-10T05:12:21.508+0000
6,0 days 01:15:10.407000,16,0 days 00:01:23.957000,7,1,null,null,0 days 00:00:29.295000,0 days 00:00:18.446000,0 days 00:00:36.216000,0 days 01:14:15.745000,0 days 01:14:34.191000,0 days 01:15:10.407000,263.0,299.0,281.0,280.0,MEDIUM,7.0,true,0 days 01:13:46.450000,Ferrari,LEC,1,false,2022-04-10T05:14:46.461+0000
7,0 days 01:16:33.996000,16,0 days 00:01:23.589000,8,1,null,null,0 days 00:00:29.198000,0 days 00:00:18.450000,0 days 00:00:35.941000,0 days 01:15:39.605000,0 days 01:15:58.055000,0 days 01:16:33.996000,265.0,294.0,282.0,284.0,MEDIUM,8.0,true,0 days 01:15:10.407000,Ferrari,LEC,1,true,2022-04-10T05:16:10.418+0000
8,0 days 01:17:57.386000,16,0 days 00:01:23.390000,9,1,null,null,0 days 00:00:28.887000,0 days 00:00:18.477000,0 days 00:00:36.026000,0 days 01:17:02.883000,0 days 01:17:21.360000,0 days 01:17:57.386000,270.0,297.0,281.0,286.0,MEDIUM,9.0,true,0 days 01:16:33.996000,Ferrari,LEC,1,true,2022-04-10T05:17:34.007+0000
9,0 days 01:19:20.746000,16,0 days 00:01:23.360000,10,1,null,null,0 days 00:00:28.936000,0 days 00:00:18.476000,0 days 00:00:35.948000,0 days 01:18:26.322000,0 days 01:18:44.798000,0 days 01:19:20.746000,272.0,300.0,282.0,283.0,MEDIUM,10.0,true,0 days 01:17:57.386000,Ferrari,LEC,1,true,2022-04-10T05:18:57.397+0000


DF COUNT: 1042


In [0]:
features = ['LapTime', "LapNumber",'Stint','Sector1Time','Sector2Time','Sector3Time','Compound','TyreLife','Team','Driver']

# Drop NULL values.
filtered_df = df.na.drop(subset = features)
print(f"DF COUNT without null values: {filtered_df.count()}")

DF COUNT without null values: 928


In [0]:
filtered_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Time: string (nullable = true)
 |-- DriverNumber: integer (nullable = true)
 |-- LapTime: string (nullable = true)
 |-- LapNumber: integer (nullable = true)
 |-- Stint: integer (nullable = true)
 |-- PitOutTime: string (nullable = true)
 |-- PitInTime: string (nullable = true)
 |-- Sector1Time: string (nullable = true)
 |-- Sector2Time: string (nullable = true)
 |-- Sector3Time: string (nullable = true)
 |-- Sector1SessionTime: string (nullable = true)
 |-- Sector2SessionTime: string (nullable = true)
 |-- Sector3SessionTime: string (nullable = true)
 |-- SpeedI1: double (nullable = true)
 |-- SpeedI2: double (nullable = true)
 |-- SpeedFL: double (nullable = true)
 |-- SpeedST: double (nullable = true)
 |-- Compound: string (nullable = true)
 |-- TyreLife: double (nullable = true)
 |-- FreshTyre: boolean (nullable = true)
 |-- LapStartTime: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- Driver: string (nullable = true)
 

In [0]:
# UDF for converting lap time into seconds.

# Return Lap Times in terms of seconds.
def time_sec(s):
  if s.startswith('0 days 00:'):
    s = s[10:]
  (splitted_time,sec) = s.split(":")

  total = float(splitted_time) * 60 + float(sec)
  answer = round(total,3)
  return answer

udf_time_sec = F.udf(time_sec, FloatType())

In [0]:
times = ['s1_time','s2_time','s3_time', 'time_sec']
df_times = ['Sector1Time','Sector2Time','Sector3Time', 'LapTime']
time_secp = udf(time_sec)

for i in range(4):
    filtered_df = filtered_df.withColumn(times[i],udf_time_sec(col(df_times[i])))

display(filtered_df)

_c0,Time,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,Compound,TyreLife,FreshTyre,LapStartTime,Team,Driver,TrackStatus,IsAccurate,LapStartDate,time_sec,s1_time,s2_time,s3_time
1,0 days 01:05:08.114000,16,0 days 00:01:24.746000,2,1,null,null,0 days 00:00:29.566000,0 days 00:00:18.608000,0 days 00:00:36.572000,0 days 01:04:12.934000,0 days 01:04:31.542000,0 days 01:05:08.114000,262.0,293.0,280.0,280.0,MEDIUM,2.0,true,0 days 01:03:43.368000,Ferrari,LEC,2,true,2022-04-10T05:04:43.379+0000,84.746,29.566,18.608,36.572
2,0 days 01:06:51.459000,16,0 days 00:01:43.345000,3,1,null,null,0 days 00:00:29.279000,0 days 00:00:24.719000,0 days 00:00:49.347000,0 days 01:05:37.393000,0 days 01:06:02.112000,0 days 01:06:51.459000,268.0,157.0,241.0,247.0,MEDIUM,3.0,true,0 days 01:05:08.114000,Ferrari,LEC,264,false,2022-04-10T05:06:08.125+0000,103.345,29.279,24.719,49.347
3,0 days 01:09:06.561000,16,0 days 00:02:15.102000,4,1,null,null,0 days 00:00:42.800000,0 days 00:00:30.843000,0 days 00:01:01.459000,0 days 01:07:34.259000,0 days 01:08:05.102000,0 days 01:09:06.561000,138.0,171.0,229.0,null,MEDIUM,4.0,true,0 days 01:06:51.459000,Ferrari,LEC,4,false,2022-04-10T05:07:51.470+0000,135.102,42.8,30.843,61.459
4,0 days 01:11:21.497000,16,0 days 00:02:14.936000,5,1,null,null,0 days 00:00:45.666000,0 days 00:00:30.394000,0 days 00:00:58.876000,0 days 01:09:52.227000,0 days 01:10:22.621000,0 days 01:11:21.497000,177.0,200.0,221.0,134.0,MEDIUM,5.0,true,0 days 01:09:06.561000,Ferrari,LEC,4,false,2022-04-10T05:10:06.572+0000,134.936,45.666,30.394,58.876
5,0 days 01:13:46.450000,16,0 days 00:02:24.953000,6,1,null,null,0 days 00:00:45.174000,0 days 00:00:31.308000,0 days 00:01:08.471000,0 days 01:12:06.671000,0 days 01:12:37.979000,0 days 01:13:46.450000,185.0,136.0,280.0,291.0,MEDIUM,6.0,true,0 days 01:11:21.497000,Ferrari,LEC,4,false,2022-04-10T05:12:21.508+0000,144.953,45.174,31.308,68.471
6,0 days 01:15:10.407000,16,0 days 00:01:23.957000,7,1,null,null,0 days 00:00:29.295000,0 days 00:00:18.446000,0 days 00:00:36.216000,0 days 01:14:15.745000,0 days 01:14:34.191000,0 days 01:15:10.407000,263.0,299.0,281.0,280.0,MEDIUM,7.0,true,0 days 01:13:46.450000,Ferrari,LEC,1,false,2022-04-10T05:14:46.461+0000,83.957,29.295,18.446,36.216
7,0 days 01:16:33.996000,16,0 days 00:01:23.589000,8,1,null,null,0 days 00:00:29.198000,0 days 00:00:18.450000,0 days 00:00:35.941000,0 days 01:15:39.605000,0 days 01:15:58.055000,0 days 01:16:33.996000,265.0,294.0,282.0,284.0,MEDIUM,8.0,true,0 days 01:15:10.407000,Ferrari,LEC,1,true,2022-04-10T05:16:10.418+0000,83.589,29.198,18.45,35.941
8,0 days 01:17:57.386000,16,0 days 00:01:23.390000,9,1,null,null,0 days 00:00:28.887000,0 days 00:00:18.477000,0 days 00:00:36.026000,0 days 01:17:02.883000,0 days 01:17:21.360000,0 days 01:17:57.386000,270.0,297.0,281.0,286.0,MEDIUM,9.0,true,0 days 01:16:33.996000,Ferrari,LEC,1,true,2022-04-10T05:17:34.007+0000,83.39,28.887,18.477,36.026
9,0 days 01:19:20.746000,16,0 days 00:01:23.360000,10,1,null,null,0 days 00:00:28.936000,0 days 00:00:18.476000,0 days 00:00:35.948000,0 days 01:18:26.322000,0 days 01:18:44.798000,0 days 01:19:20.746000,272.0,300.0,282.0,283.0,MEDIUM,10.0,true,0 days 01:17:57.386000,Ferrari,LEC,1,true,2022-04-10T05:18:57.397+0000,83.36,28.936,18.476,35.948
10,0 days 01:20:44.102000,16,0 days 00:01:23.356000,11,1,null,null,0 days 00:00:28.983000,0 days 00:00:18.444000,0 days 00:00:35.929000,0 days 01:19:49.729000,0 days 01:20:08.173000,0 days 01:20:44.102000,268.0,299.0,281.0,281.0,MEDIUM,11.0,true,0 days 01:19:20.746000,Ferrari,LEC,1,true,2022-04-10T05:20:20.757+0000,83.356,28.983,18.444,35.929


In [0]:
filtered_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Time: string (nullable = true)
 |-- DriverNumber: integer (nullable = true)
 |-- LapTime: string (nullable = true)
 |-- LapNumber: integer (nullable = true)
 |-- Stint: integer (nullable = true)
 |-- PitOutTime: string (nullable = true)
 |-- PitInTime: string (nullable = true)
 |-- Sector1Time: string (nullable = true)
 |-- Sector2Time: string (nullable = true)
 |-- Sector3Time: string (nullable = true)
 |-- Sector1SessionTime: string (nullable = true)
 |-- Sector2SessionTime: string (nullable = true)
 |-- Sector3SessionTime: string (nullable = true)
 |-- SpeedI1: double (nullable = true)
 |-- SpeedI2: double (nullable = true)
 |-- SpeedFL: double (nullable = true)
 |-- SpeedST: double (nullable = true)
 |-- Compound: string (nullable = true)
 |-- TyreLife: double (nullable = true)
 |-- FreshTyre: boolean (nullable = true)
 |-- LapStartTime: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- Driver: string (nullable = true)
 

In [0]:
# String indexing string features.
string_features = ["Driver","Team","Compound"]


for ele in string_features:
    indexer = StringIndexer(inputCol = ele, outputCol = ele + '_indexed')
    filtered_df = indexer.fit(filtered_df).transform(filtered_df)

    
display(filtered_df)

_c0,Time,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,Compound,TyreLife,FreshTyre,LapStartTime,Team,Driver,TrackStatus,IsAccurate,LapStartDate,time_sec,s1_time,s2_time,s3_time,Driver_indexed,Team_indexed,Compound_indexed
1,0 days 01:05:08.114000,16,0 days 00:01:24.746000,2,1,null,null,0 days 00:00:29.566000,0 days 00:00:18.608000,0 days 00:00:36.572000,0 days 01:04:12.934000,0 days 01:04:31.542000,0 days 01:05:08.114000,262.0,293.0,280.0,280.0,MEDIUM,2.0,true,0 days 01:03:43.368000,Ferrari,LEC,2,true,2022-04-10T05:04:43.379+0000,84.746,29.566,18.608,36.572,9.0,9.0,1.0
2,0 days 01:06:51.459000,16,0 days 00:01:43.345000,3,1,null,null,0 days 00:00:29.279000,0 days 00:00:24.719000,0 days 00:00:49.347000,0 days 01:05:37.393000,0 days 01:06:02.112000,0 days 01:06:51.459000,268.0,157.0,241.0,247.0,MEDIUM,3.0,true,0 days 01:05:08.114000,Ferrari,LEC,264,false,2022-04-10T05:06:08.125+0000,103.345,29.279,24.719,49.347,9.0,9.0,1.0
3,0 days 01:09:06.561000,16,0 days 00:02:15.102000,4,1,null,null,0 days 00:00:42.800000,0 days 00:00:30.843000,0 days 00:01:01.459000,0 days 01:07:34.259000,0 days 01:08:05.102000,0 days 01:09:06.561000,138.0,171.0,229.0,null,MEDIUM,4.0,true,0 days 01:06:51.459000,Ferrari,LEC,4,false,2022-04-10T05:07:51.470+0000,135.102,42.8,30.843,61.459,9.0,9.0,1.0
4,0 days 01:11:21.497000,16,0 days 00:02:14.936000,5,1,null,null,0 days 00:00:45.666000,0 days 00:00:30.394000,0 days 00:00:58.876000,0 days 01:09:52.227000,0 days 01:10:22.621000,0 days 01:11:21.497000,177.0,200.0,221.0,134.0,MEDIUM,5.0,true,0 days 01:09:06.561000,Ferrari,LEC,4,false,2022-04-10T05:10:06.572+0000,134.936,45.666,30.394,58.876,9.0,9.0,1.0
5,0 days 01:13:46.450000,16,0 days 00:02:24.953000,6,1,null,null,0 days 00:00:45.174000,0 days 00:00:31.308000,0 days 00:01:08.471000,0 days 01:12:06.671000,0 days 01:12:37.979000,0 days 01:13:46.450000,185.0,136.0,280.0,291.0,MEDIUM,6.0,true,0 days 01:11:21.497000,Ferrari,LEC,4,false,2022-04-10T05:12:21.508+0000,144.953,45.174,31.308,68.471,9.0,9.0,1.0
6,0 days 01:15:10.407000,16,0 days 00:01:23.957000,7,1,null,null,0 days 00:00:29.295000,0 days 00:00:18.446000,0 days 00:00:36.216000,0 days 01:14:15.745000,0 days 01:14:34.191000,0 days 01:15:10.407000,263.0,299.0,281.0,280.0,MEDIUM,7.0,true,0 days 01:13:46.450000,Ferrari,LEC,1,false,2022-04-10T05:14:46.461+0000,83.957,29.295,18.446,36.216,9.0,9.0,1.0
7,0 days 01:16:33.996000,16,0 days 00:01:23.589000,8,1,null,null,0 days 00:00:29.198000,0 days 00:00:18.450000,0 days 00:00:35.941000,0 days 01:15:39.605000,0 days 01:15:58.055000,0 days 01:16:33.996000,265.0,294.0,282.0,284.0,MEDIUM,8.0,true,0 days 01:15:10.407000,Ferrari,LEC,1,true,2022-04-10T05:16:10.418+0000,83.589,29.198,18.45,35.941,9.0,9.0,1.0
8,0 days 01:17:57.386000,16,0 days 00:01:23.390000,9,1,null,null,0 days 00:00:28.887000,0 days 00:00:18.477000,0 days 00:00:36.026000,0 days 01:17:02.883000,0 days 01:17:21.360000,0 days 01:17:57.386000,270.0,297.0,281.0,286.0,MEDIUM,9.0,true,0 days 01:16:33.996000,Ferrari,LEC,1,true,2022-04-10T05:17:34.007+0000,83.39,28.887,18.477,36.026,9.0,9.0,1.0
9,0 days 01:19:20.746000,16,0 days 00:01:23.360000,10,1,null,null,0 days 00:00:28.936000,0 days 00:00:18.476000,0 days 00:00:35.948000,0 days 01:18:26.322000,0 days 01:18:44.798000,0 days 01:19:20.746000,272.0,300.0,282.0,283.0,MEDIUM,10.0,true,0 days 01:17:57.386000,Ferrari,LEC,1,true,2022-04-10T05:18:57.397+0000,83.36,28.936,18.476,35.948,9.0,9.0,1.0
10,0 days 01:20:44.102000,16,0 days 00:01:23.356000,11,1,null,null,0 days 00:00:28.983000,0 days 00:00:18.444000,0 days 00:00:35.929000,0 days 01:19:49.729000,0 days 01:20:08.173000,0 days 01:20:44.102000,268.0,299.0,281.0,281.0,MEDIUM,11.0,true,0 days 01:19:20.746000,Ferrari,LEC,1,true,2022-04-10T05:20:20.757+0000,83.356,28.983,18.444,35.929,9.0,9.0,1.0


In [0]:
# Final Features to be used.

final_features = ['time_sec','Team_indexed', 'Driver_indexed', 'LapNumber', 's1_time','s2_time','s3_time', 'Stint','Compound_indexed','TyreLife']

featureVectorDF = filtered_df.select(final_features)
featureVectorDF = featureVectorDF.withColumnRenamed('time_sec','label')
display(featureVectorDF)

label,Team_indexed,Driver_indexed,LapNumber,s1_time,s2_time,s3_time,Stint,Compound_indexed,TyreLife
84.746,9.0,9.0,2,29.566,18.608,36.572,1,1.0,2.0
103.345,9.0,9.0,3,29.279,24.719,49.347,1,1.0,3.0
135.102,9.0,9.0,4,42.8,30.843,61.459,1,1.0,4.0
134.936,9.0,9.0,5,45.666,30.394,58.876,1,1.0,5.0
144.953,9.0,9.0,6,45.174,31.308,68.471,1,1.0,6.0
83.957,9.0,9.0,7,29.295,18.446,36.216,1,1.0,7.0
83.589,9.0,9.0,8,29.198,18.45,35.941,1,1.0,8.0
83.39,9.0,9.0,9,28.887,18.477,36.026,1,1.0,9.0
83.36,9.0,9.0,10,28.936,18.476,35.948,1,1.0,10.0
83.356,9.0,9.0,11,28.983,18.444,35.929,1,1.0,11.0


In [0]:
featureVectorDF.printSchema()

root
 |-- label: float (nullable = true)
 |-- Team_indexed: double (nullable = false)
 |-- Driver_indexed: double (nullable = false)
 |-- LapNumber: integer (nullable = true)
 |-- s1_time: float (nullable = true)
 |-- s2_time: float (nullable = true)
 |-- s3_time: float (nullable = true)
 |-- Stint: integer (nullable = true)
 |-- Compound_indexed: double (nullable = false)
 |-- TyreLife: double (nullable = true)



In [0]:
for i in featureVectorDF.columns:
    print(f"corr of {i}: {featureVectorDF.stat.corr('label', i)}")


corr of label: 1.0
corr of Team_indexed: -0.0030509615643234448
corr of Driver_indexed: 0.018725561183712287
corr of LapNumber: -0.38309232681989186
corr of s1_time: 0.9111193229715138
corr of s2_time: 0.9409869696488421
corr of s3_time: 0.9685767499069436
corr of Stint: -0.15635049223781314
corr of Compound_indexed: 0.13552797716827472
corr of TyreLife: -0.3593841857510368


In [0]:
vectorAssembler = VectorAssembler(inputCols = ['Team_indexed', 'Driver_indexed', 'LapNumber', 's1_time','s2_time','s3_time', 'Stint','Compound_indexed','TyreLife'], outputCol = 'features')
vdf = vectorAssembler.transform(featureVectorDF)
vhouse_df = vdf.select(['features','label'])
vhouse_df.show(5)

+--------------------+-------+
|            features|  label|
+--------------------+-------+
|[9.0,9.0,2.0,29.5...| 84.746|
|[9.0,9.0,3.0,29.2...|103.345|
|[9.0,9.0,4.0,42.7...|135.102|
|[9.0,9.0,5.0,45.6...|134.936|
|[9.0,9.0,6.0,45.1...|144.953|
+--------------------+-------+
only showing top 5 rows



In [0]:
splits = vdf.randomSplit([0.7,0.3])
train_df = splits[0]
test_df = splits[1]


In [0]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='label', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.0,0.0,0.0,0.9756763060307347,0.994692570202715,0.9792845657053972,0.0,0.0,0.0]
Intercept: 1.6872099908835403


In [0]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","label","features").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="label",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+-----------------+------+--------------------+
|       prediction| label|            features|
+-----------------+------+--------------------+
|80.45886559820252| 80.26|[9.0,9.0,58.0,28....|
| 81.1524454674424|80.966|[9.0,9.0,49.0,28....|
|81.20259187699547|81.017|[9.0,9.0,50.0,28....|
| 81.6041180842871|81.427|[9.0,9.0,45.0,28....|
|81.66644657479544|81.489|[9.0,9.0,48.0,28....|
+-----------------+------+--------------------+
only showing top 5 rows

R Squared (R2) on test data = 0.999676


In [0]:
predictions = lr_model.transform(test_df)
predictions.select("prediction","label","features").show(50)

+-----------------+------+--------------------+
|       prediction| label|            features|
+-----------------+------+--------------------+
|80.45886559820252| 80.26|[9.0,9.0,58.0,28....|
| 81.1524454674424|80.966|[9.0,9.0,49.0,28....|
|81.20259187699547|81.017|[9.0,9.0,50.0,28....|
| 81.6041180842871|81.427|[9.0,9.0,45.0,28....|
|81.66644657479544|81.489|[9.0,9.0,48.0,28....|
|81.67882671749875|81.504|[9.0,9.0,51.0,28....|
|81.79952973267437|81.626|[9.0,9.0,46.0,28....|
|81.86230686967149| 81.69|[5.0,4.0,57.0,28....|
| 82.0565155979131|81.887|[1.0,6.0,49.0,28....|
|82.05426599911254|81.887|[5.0,4.0,48.0,28....|
|82.07743955972025|81.911|[5.0,4.0,55.0,28....|
|82.09097724799352|81.924|[5.0,4.0,49.0,28....|
|82.09586496834557| 81.93|[5.0,4.0,44.0,28....|
|82.13735364467912|81.972|[1.0,8.0,49.0,28....|
| 82.2005778551112|82.035|[5.0,4.0,43.0,28....|
|82.20289374568031|82.036|[1.0,8.0,46.0,28....|
|82.21505498083113|82.049|[1.0,8.0,42.0,28....|
| 82.2615438967482|  82.1|[9.0,9.0,57.0,